In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

Definitions: A quarter is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December. 

A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth. A recession bottom is the quarter within a recession which had the lowest GDP. 
A university town is a city which has a high percentage of university students compared to the total population of the city. 

Hypothesis: University towns have their mean housing prices less effected by recessions. 

We would Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    df=pd.read_csv('university_towns.txt',sep='\n',header=None,names= ['raw data'])

    df1 = pd.DataFrame(columns=[ "State","RegionName"])
    index=len(df.index)
    for x in range(index):
        if (df.loc[x,'raw data'])[-6:]=='[edit]':
            state=(df.loc[x,'raw data'])[:-6]
            continue
        b=df.loc[x,'raw data']
        word=''
        for y in b:
            if y=='[' or y=='(':
                word=word.strip()
                break
            word=word+y
        
        b=pd.Series([state,word],index=['State','RegionName'])    
    
        df1=df1.append(b,ignore_index=True)
    

    

        
    
    
    
    return df1

In [11]:
get_list_of_university_towns().head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [4]:
def get_recession_start():
    df=pd.read_excel('gdplev.xls')
    df=df[['Unnamed: 4','Unnamed: 6']]
    df.rename(columns={'Unnamed: 4':'Year','Unnamed: 6':'GDP'},inplace=True)

    df=df.set_index('Year')
    df=df.loc['2000q1':]
    df.reset_index(inplace=True)
    for i in range(2,len(df.index)):
    
        if(df.loc[i,'GDP'] -df.loc[i-1,'GDP'])<0 and (df.loc[i-1,'GDP']-df.loc[i-2,'GDP'])<0:
            a=df.loc[i-1,'Year']
            break

               
               

    
    return a

In [12]:
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    df=pd.read_excel('gdplev.xls')
    df=df[['Unnamed: 4','Unnamed: 6']]
    df.rename(columns={'Unnamed: 4':'Year','Unnamed: 6':'GDP'},inplace=True)

    df=df.set_index('Year')
    df=df.loc['2000q1':]
    df.reset_index(inplace=True)
    for i in range(2,len(df.index)):
    
        if(df.loc[i,'GDP'] -df.loc[i-1,'GDP'])<0 and (df.loc[i-1,'GDP']-df.loc[i-2,'GDP'])<0:
            for x in range(i+2,len(df.index)):
                if(df.loc[x,'GDP']-df.loc[x-1,'GDP'])>0 and (df.loc[x-1,'GDP']-df.loc[x-2,'GDP'])>0:
                    a=df.loc[x,'Year']
                    break
            break
        else:
            pass
    

    

             
               

       
    return a

In [13]:
get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
        df=pd.read_excel('gdplev.xls')
        df=df[['Unnamed: 4','Unnamed: 6']]
        df.rename(columns={'Unnamed: 4':'Year','Unnamed: 6':'GDP'},inplace=True)

        df=df.set_index('Year')
        df=df.loc['2000q1':]
        df.reset_index(inplace=True)
        lst=list()
        for i in range(2,len(df.index)):
    
            if(df.loc[i,'GDP'] -df.loc[i-1,'GDP'])<0 and (df.loc[i-1,'GDP']-df.loc[i-2,'GDP'])<0:
                for x in range(i+2,len(df.index)):
                    if(df.loc[x,'GDP']-df.loc[x-1,'GDP'])>0 and (df.loc[x-1,'GDP']-df.loc[x-2,'GDP']):
                        lst.append(df.loc[i:x,'GDP'].min())
                    
                    
                    break
                    
                    
        df= df[df['GDP']== min(lst)] 
    
        return df.iloc[0]['Year']

In [14]:
get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    df=pd.read_csv('City_Zhvi_AllHomes.csv')

    for i in range(len(df.columns)):
        if df.columns[i]=='2000-01':
            break
    col=list(df.columns[i:])
    col=['State','RegionName']+col
    df=df[col]
    def bak(row):
        row['State']=states[row['State']]
        return row['State']
    
    a=df.apply(bak,axis=1)
    df['State']=a
    df.set_index(['State','RegionName'],inplace=True)
    df.columns = pd.to_datetime(df.columns)
    df=df.resample('Q',axis=1).mean()
    per=df.columns.to_period()
    df.columns=(per)
    df.columns=df.columns.to_series().astype(str)
    df.columns=df.columns.map(str.lower)

    
    
    
    return df

In [15]:
convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [9]:
def run_ttest():
    hdf=convert_housing_data_to_quarters()
    start=get_recession_start()

    i=(list(hdf.columns)).index(start)
    rec_start=hdf.columns[i-1]
    rec_bottom = get_recession_bottom()
    hdf['PriceRatio']=hdf[rec_start]/hdf[rec_bottom]
    hdf=hdf['PriceRatio']
    hdf.dropna(inplace=True)
    df1=get_list_of_university_towns()
    a=df1.set_index(['State','RegionName'])
    i=list(a.index)
    i2=list(hdf.index)
    hdf1=hdf.loc[i]
    hdf1.dropna(inplace=True)
    for x in i:
        if i in i2:
            i2.remove(x)
        
        
    hdf2=(hdf.loc[i2])
    if (ttest_ind(hdf1,hdf2))[1] <0.01:
        diff=True
    else:
        diff=False
    
    if hdf1.mean() < hdf2.mean():
        better="university town"
    else:
        better='"non-university town"'
    
    return (diff,(ttest_ind(hdf1,hdf2))[1],better)

In [10]:
run_ttest()

(True, 0.0074890107104271485, 'university town')